In [ ]:
import pandas as pd
import feather

#######################################################
###################### READ CSV #######################

df = pd.read_csv('./all/train.csv', parse_dates=['pickup_datetime'])

#######################################################
##################WRITE FEATHER #######################

df.to_feather('./all/train.feather')

#######################################################
##################### READ FEATHER ####################

#df = pd.read_feather('./all/train.feather')

#######################################################
############## WRITE PREPROCESSED FEATHER #############

#df.to_feather('./all/preprocessed_train.feather')

#######################################################
######### READ PREPROCESSED FEATHER ###################

#df = pd.read_feather('./all/preprocessed_train.feather')